In [1]:
import cv2
import numpy as np
import os 
import joblib
import face_recognition
from imutils import paths
import imutils

In [2]:
#Load Trained Model
data=joblib.load('trained_model.pkl')

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if(h>w and h>800):
        height=800
    elif(w>800):
        width=800
        
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    #print(dim)
    #if (dim[0] >800):
        #dim=(int(dim[0]/2),int(dim[1]/2))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [4]:
#Testing from image path
def test_from_path(path):
    image =cv2.imread(path)
    image = image_resize(image)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    boxes = face_recognition.face_locations(rgb)
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    confidences=[]
    
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"],encoding)
        name = "Unknown"
        conf=0
        #print(matches)
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {} 
            conf=face_recognition.face_distance(data["encodings"],encoding).mean()*100
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get)
            if(counts[name]<=5):
                name="Unknown"
                conf=0
        names.append(name)
        confidences.append(conf)
    print(names)
    while True:
        for ((top, right, bottom, left), name,conf) in zip(boxes, names,confidences):
            # draw the predicted face name on the image
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                0.75, (255, 255, 0), 2)
            cv2.putText(image, str(round(conf,2))+"%", (left+5,bottom-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
        # show the output image
        cv2.imshow('image', image)
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            #cv2.imwrite('Res/test_res_'+path.split('/')[-1]+str(names)+'.png', image)
            break
    cv2.destroyAllWindows()


In [5]:
#testing from webcam
def test_from_webcam():
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480) 
    si=0
    while True:

        ret, image =cam.read()
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes = face_recognition.face_locations(rgb)
        encodings = face_recognition.face_encodings(rgb, boxes)
        names = []

        # loop over the facial encoding
        for encoding in encodings:
            matches = face_recognition.compare_faces(data["encodings"],
                encoding)
            #print(matches)
            name = "Unknown"
            
            # check to see if we have found a match
            if True in matches:
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                #print(matchedIdxs)
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1
                # determine the recognized face with the largest number of votes
                
                name = max(counts, key=counts.get)
                if(counts[name]<=2):
                    name="Unknown"
                names.append(name)
            
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # draw the predicted face name on the image
            cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                0.75, (255,255,255), 2)  

        cv2.imshow('camera',image) 
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video and 's' to save res
        if k == 27:            
            break
        elif k==ord('s'):
            cv2.imwrite('res/'+'res'+str(si)+'.png', image)
            si+=1
    cam.release()
    cv2.destroyAllWindows()

In [6]:
imagePaths = list(paths.list_images('testing'))
for (i, imagePath) in enumerate(imagePaths):
    print(imagePath.split(os.path.sep)[-1],end="-")
    test_from_path(imagePath)#Press Esc on your keyboard to next image
    print()
    

test1.jpg-['Evans', 'Rdj']

test10.jpg-['Unknown', 'Unknown', 'Unknown', 'Rdj', 'Evans', 'Unknown', 'Unknown']

test2.jpg-['Rdj', 'Unknown', 'Evans']

test4.jpg-['Unknown', 'Evans', 'Unknown', 'Evans', 'Rdj', 'Unknown', 'Rdj']

test5.jpg-['Rdj', 'Unknown', 'Unknown', 'Unknown', 'Rdj', 'Evans', 'Unknown', 'Evans']

test7.jpg-['Rdj', 'Unknown', 'Unknown', 'Unknown', 'Evans', 'Unknown', 'Unknown']

test8.jpeg-['Unknown', 'Unknown', 'Rdj']

test9.jpg-['Rdj', 'Evans']

